***
# TEST
***

In [ ]:
import os
import json
import pandas as pd
import pvdeg
from pytest import approx
from pvdeg import TEST_DATA_DIR

#Load weather data
WEATHER = pd.read_csv(os.path.join(TEST_DATA_DIR, 'weather_day_pytest.csv'),
                         index_col= 0, parse_dates=True)
with open(os.path.join(TEST_DATA_DIR, 'meta.json'),'r') as file:
    META = json.load(file)

#Load expected results
rh_expected = pd.read_csv(os.path.join(TEST_DATA_DIR, 'input_day_pytest.csv'),
                          index_col=0, parse_dates=True)
rh_cols = [col for col in rh_expected.columns if 'RH' in col]
rh_expected = rh_expected[rh_cols]


def test_module():
    '''
    test pvdeg.humidity.calc_rel_humidity
    
    Requires:
    ---------
    weather dataframe and meta dictionary
    '''
    result = pvdeg.humidity.module(WEATHER, META)
    pd.testing.assert_frame_equal(result, 
                                  rh_expected, 
                                  check_dtype=False)

def test_psat():
    '''
    test pvdeg.humidity.psat
    
    Requires:
    ---------
    weahter dataframe and meta dictionary
    '''
    psat_avg = pvdeg.humidity.psat(temp=WEATHER['temp_air'])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)



In [ ]:
test_psat()

In [1]:
import pvdeg
import pickle
import pandas as pd
import numpy as np
import os
from pvdeg import TEST_DATA_DIR

with open(r"C:\Users\tford\Downloads\summit-weather.pkl", 'rb') as f:
    GEO_WEATHER = pickle.load(f)

GEO_META = pd.read_csv(r"C:\Users\tford\Downloads\summit-meta.csv", index_col=0)

In [2]:
GEO_WEATHER

<xarray.Dataset> Size: 9MB
Dimensions:            (time: 17520, gid: 11)
Coordinates:
  * gid                (gid) int64 88B 449211 452064 453020 ... 460613 462498
  * time               (time) datetime64[ns] 140kB 2022-01-01 ... 2022-12-31T...
Data variables:
    temp_air           (time, gid) float64 2MB -12.0 -8.1 -14.2 ... -4.3 -6.2
    wind_speed         (time, gid) float64 2MB 0.6 0.6 0.3 0.6 ... 0.9 1.0 1.1
    dhi                (time, gid) float64 2MB 0.0 0.0 0.0 ... 13.0 18.0 19.0
    ghi                (time, gid) float64 2MB 0.0 0.0 0.0 ... 13.0 24.0 19.0
    dni                (time, gid) float64 2MB 0.0 0.0 0.0 0.0 ... 0.0 126.0 1.0
    relative_humidity  (time, gid) float64 2MB 99.93 79.41 100.0 ... 95.93 100.0
Attributes:
    full_version_record:  {"rex": "0.2.80", "pandas": "2.0.0", "numpy": "1.23...
    package:              rex
    version:              4.0.0

In [3]:
GEO_META

,latitude,longitude,altitude,tz,country,state,county,wind_height
449211,39.89,-106.42,2954,-7,United States,Colorado,Summit,2
452064,39.89,-106.30,2486,-7,United States,Colorado,Summit,2
453020,39.69,-106.26,3485,-7,United States,Colorado,Summit,2
454916,39.81,-106.18,2767,-7,United States,Colorado,Summit,2
455867,39.81,-106.14,2883,-7,United States,Colorado,Summit,2
455877,39.41,-106.14,3859,-7,United States,Colorado,Summit,2
456827,39.45,-106.10,3634,-7,United States,Colorado,Summit,2
457776,39.41,-106.06,3363,-7,United States,Colorado,Summit,2
459670,39.65,-105.98,3261,-7,United States,Colorado,Summit,2
460613,39.53,-105.94,3190,-7,United States,Colorado,Summit,2


In [18]:
res = pvdeg.geospatial.analysis(
    weather_ds=GEO_WEATHER,
    meta_df=GEO_META,
    func=pvdeg.standards.standoff,
)

The array tilt angle was not provided, therefore the latitude tilt of 39.9 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.9 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.8 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.8 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.4 was used.
The array azimuth was not provided, therefore an azimuth of 180.0 was used.
The array tilt angle was not provi

In [6]:
data_var = res["x"]

# Stack the latitude and longitude coordinates into a single dimension
stacked = data_var.stack(z=("latitude", "longitude"))

# Extract the coordinates as numpy arrays
latitudes = stacked['latitude'].values
longitudes = stacked['longitude'].values

# Extract the data values
data_values = stacked.values

# Combine coordinates and data values into a single numpy array
combined_array = np.column_stack((latitudes, longitudes, data_values))

print(combined_array)

[[ 3.94100000e+01 -1.06420000e+02             nan]
 [ 3.94100000e+01 -1.06300000e+02             nan]
 [ 3.94100000e+01 -1.06260000e+02             nan]
 [ 3.94100000e+01 -1.06180000e+02             nan]
 [ 3.94100000e+01 -1.06140000e+02  5.05368226e-01]
 [ 3.94100000e+01 -1.06100000e+02             nan]
 [ 3.94100000e+01 -1.06060000e+02  4.23099392e-01]
 [ 3.94100000e+01 -1.05980000e+02             nan]
 [ 3.94100000e+01 -1.05940000e+02             nan]
 [ 3.94100000e+01 -1.05860000e+02             nan]
 [ 3.94500000e+01 -1.06420000e+02             nan]
 [ 3.94500000e+01 -1.06300000e+02             nan]
 [ 3.94500000e+01 -1.06260000e+02             nan]
 [ 3.94500000e+01 -1.06180000e+02             nan]
 [ 3.94500000e+01 -1.06140000e+02             nan]
 [ 3.94500000e+01 -1.06100000e+02  4.74997232e-01]
 [ 3.94500000e+01 -1.06060000e+02             nan]
 [ 3.94500000e+01 -1.05980000e+02             nan]
 [ 3.94500000e+01 -1.05940000e+02             nan]
 [ 3.94500000e+01 -1.05860000e+

In [13]:
res = pd.DataFrame(combined_array).dropna()
ans = pd.read_csv(os.path.join(TEST_DATA_DIR, 'summit-standoff-res.csv'), index_col=0)
res.columns = ans.columns

In [17]:
pd.testing.assert_frame_equal(res, ans, check_dtype=False, check_names=False)